In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Gemini 2.5 Pro to develop MCP Server



<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  
  
  
  <td style="text-align: center">
    <a href="https://github.com/wadave/vertex_ai_mcp_samples/blob/main/create_mcp_server_by_gemini.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author(s) |
| --- |
| [Dave Wang](https://github.com/wadave) |

## Overview
The Model Context Protocol (MCP) is an open standard that simplifies how AI assistants connect with external data, tools, and systems. It achieves this by standardizing the way applications provide contextual information to Large Language Models (LLMs), creating a vital interface for models to interact directly with various external services.

Developers building MCP-enabled applications have the flexibility to utilize existing third-party MCP servers or implement their own custom server solutions.

This notebook focuses on the latter, demonstrating how to build custom MCP servers using Gemini 2.5 Pro. We will walk through code generation and testing for three specific examples:

#### MCP server code generation:
- Example 1: Creating a BigQuery MCP Server
- Example 2: Creating a MedlinePlus MCP Server
- Example 3: Creating an NIH MCP Server

#### MCP server code testing:
- Option 1: Use LangChain MCP Adaptor
- Option 2: Build your own agent

## Get started

### Install Google Gen AI SDK and other required packages


In [29]:
%pip install --upgrade --quiet google-genai google-cloud-secret-manager mcp geopy black google-cloud-bigquery langchain-mcp-adapters langchain langchain-google-vertexai langgraph

Note: you may need to restart the kernel to use updated packages.


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import Libraries

In [1]:
from google.cloud import secretmanager
from google import genai
import datetime

from google.genai.types import (
    GenerateContentConfig,
)

from typing import List, Dict, Any
import json
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio
from google.genai import types
from typing import List

import sys
import os
from google.cloud import aiplatform

# Create server parameters for stdio connection

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_google_vertexai import ChatVertexAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from IPython.display import display, Markdown
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage

sys.path.append(os.path.abspath("/util"))
from util.util import access_secret_version, get_url_content, format_python

### Set up Vertex AI 

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [34]:
# Use the environment variable if the user doesn't provide Project ID.
import os
from google import genai

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

## Set up Gemini client

In [9]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

MODEL_ID = (
    "gemini-2.5-pro-exp-03-25"  # This model is only for MCP server code generation.
)

### Get system instruction context info

In [4]:
# The URL you want to fetch
url = "https://modelcontextprotocol.io/quickstart/server"

In [5]:
reference_content = get_url_content(url)

Successfully fetched content


### Set up system instruction

In [35]:
from pydantic import BaseModel


class ResponseSchema(BaseModel):
    python_code: str
    description: str


system_instruction = f"""
  You are an MCP server export.
  Your mission is to write python code for MCP server.
  Here's the MCP server development guide and example
  {reference_content}
  
"""

#### Set function to generate MCP server code

In [ ]:
def generate_mcp_server(prompt):
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=prompt,
        config=GenerateContentConfig(
            system_instruction=system_instruction,
            response_mime_type="application/json",
            response_schema=ResponseSchema,
        ),
    )

    return response.text

## Generate MCP Server Code

### Example 1:  Create MCP Server for Google Cloud BigQuery

In [10]:
prompt = """
  Please create an MCP server code for google cloud big query. It has two tools. One is to list tables for all datasets, the other is to describe a table. Google cloud project id and location will be provided for use. please use project id to access big query client.
  
  Return Python code within a JSON object formatted exactly as: {'python_code':    'your_generated_code', 'description':'your description'}
"""

In [11]:
response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

In [20]:
format_python(python_code, "server/bq.py")

Successfully formatted the code and saved it to 'server/bq.py'


### Example 2:  Create MCP server for Medlineplus website
Create an MCP server for 
https://medlineplus.gov/about/developers/webservices/ API service

In [12]:
med_url = "https://medlineplus.gov/about/developers/webservices/"
med_api_details = get_url_content(med_url)

Successfully fetched content


In [14]:
prompt = f"""
  Please create an MCP server code for https://medlineplus.gov/about/developers/webservices/. It has one tool, get_medical_term. You provide a medical term, this tool will return explanation of the medial term
  
  Here's the API details:
  {med_api_details}
"""

response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

format_python(python_code, "server/med.py")

Successfully formatted the code and saved it to 'server/med.py'


### Example 3: Create MCP Server for NIH

In [13]:
nih_url = "https://clinicaltables.nlm.nih.gov/apidoc/icd10cm/v3/doc.html"
nih_api_details = get_url_content(nih_url)

Successfully fetched content


In [14]:
prompt = f"""
  Please create an MCP server code for NIH. It has one tool, get_icd_10_code. You provide a name or code, it will return top 5 results. 
  
  Here's the API details:
   {nih_api_details}
"""

response_text = generate_mcp_server(prompt)
python_code = json.loads(response_text)["python_code"]

format_python(python_code, "server/nih.py")

Successfully formatted the code and saved it to 'server/nih.py'


## Testing MCP Servers

### Option 1: Use LangChain MCP adaptor to test MCP servers

In [15]:
aiplatform.init(
    project=PROJECT_ID,
    location=LOCATION,
)

In [16]:
llm = ChatVertexAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    max_retries=6,
    stop=None,
)

In [ ]:
server_configs = {
    "nih": {
        "command": "python",
        "args": ["./server/nih.py"],
        "transport": "stdio",
    },
    "med": {
        "command": "python",
        "args": ["./server/med.py"],
        "transport": "stdio",
    },
    "bq": {
        "command": "python",
        "args": ["./server/bq.py"],
        "transport": "stdio",
    },
}

Set up MCP Client using LangChain MCP Adaptor 

In [ ]:
async def run_lc_react_agent(server_configs, message):
    async with MultiServerMCPClient(server_configs) as client:
        agent = create_react_agent(llm, client.get_tools())
        
        agent_response = await agent.ainvoke({"messages": message})
        for response in agent_response["messages"]:
            user = ""

            if isinstance(response, HumanMessage):
                user = "[User]"
            elif isinstance(response, ToolMessage):
                user = "-Tool-"
            elif isinstance(response, AIMessage):
                user = "[Agent]"

            if isinstance(response.content, list):
                display(Markdown(f'{user}: {response.content[0].get("text", "")}'))
                continue
            display(Markdown(f"{user}: {response.content}"))

#### Example 1: BigQuery MCP server testing

In [ ]:
await run_lc_react_agent(
    server_configs, "Please list my bigquery tables for project 'dw-genai-dev'"
)

[User]: Please list my bigquery tables for project 'dw-genai-dev'

[Agent]: 

-Tool-: demo_dataset1.item_table
demo_dataset1.user_table
demo_dataset2.item_table
demo_dataset2.user_table

[Agent]: OK. I found these tables in project 'dw-genai-dev':
demo_dataset1.item_table
demo_dataset1.user_table
demo_dataset2.item_table
demo_dataset2.user_table

#### Example 2: MedlinePlus MCP server testing

In [ ]:
await run_lc_react_agent(server_configs, "Please explain flu in details")

[User]: Please explain flu in details

[Agent]: 

-Tool-: What is the <span class="qt0">flu</span>?<p>The <span class="qt0">flu</span>, also called <span class="qt0">influenza</span>, is a respiratory infection caused by viruses. Each year, millions of Americans get sick with the <span class="qt0">flu</span>. Sometimes it causes mild illness. But it can also be serious or even deadly, especially for people over 65, newborn babies, and people with certain chronic illnesses.</p>What causes the <span class="qt0">flu</span>?<p>The <span class="qt0">flu</span> is caused by <span class="qt0">flu</span> viruses that spread from person to person. When someone with the <span class="qt0">flu</span> coughs, sneezes, or talks, they spray tiny droplets. These droplets can land in the mouths or noses of people who are nearby. Less often, a person may get <span class="qt0">flu</span> by touching a surface or object that has <span class="qt0">flu</span> virus on it and then touching their own mouth, nose, or possibly their eyes.</p>What are the symptoms of the <span class="qt0">flu</span>?<p>Symptoms of the <span class="qt0">flu</span> come on suddenly and may include:</p><ul><li>Fever or feeling feverish/chills</li><li>Cough</li><li>Sore throat</li><li>Runny or stuffy nose</li><li>Muscle or body aches</li><li>Headaches</li><li>Fatigue (tiredness)</li></ul><p>Some people may also have vomiting and diarrhea. This is more common in children.</p><p>Sometimes people have trouble figuring out whether they have a cold or the <span class="qt0">flu</span>. There are differences between them:</p>Signs and SymptomsColdFluStart of symptomsSlowlySuddenlyFeverRarelyUsuallyAchesSometimes (slight)UsuallyFatigue, weaknessSometimesUsuallyHeadacheRarelyCommonStuffy nose, sneezing, or sore throatCommonSometimes<p>Sometimes people say that they have a <span class="qt0">"flu"</span> when they really have something else. For example, <span class="qt0">"stomach flu"</span> isn't the <span class="qt0">flu</span>; it's gastroenteritis.</p>What other problems can the <span class="qt0">flu</span> cause?<p>Some people who get the <span class="qt0">flu</span> will develop complications. Some of these complications can be serious or even life-threatening. They include:</p><ul><li>Bronchitis</li><li>Ear infection</li><li>Sinus infection</li><li>Pneumonia</li><li>Inflammation of the heart (myocarditis), brain (encephalitis), or muscle tissues (myositis, rhabdomyolysis)</li></ul><p>The <span class="qt0">flu</span> also can make chronic health problems worse. For example, people with asthma may have asthma attacks while they have <span class="qt0">flu</span>.</p><p>Certain people are more likely to have complications from the <span class="qt0">flu</span>, including:</p><ul><li>Adults 65 and older</li><li>Pregnant women</li><li>Children younger than 5</li><li>People with certain chronic health conditions, such as asthma, diabetes, and heart disease</li></ul>How is the <span class="qt0">flu</span> diagnosed?<p>To diagnose the <span class="qt0">flu</span>, health care providers will first do a medical history and ask about your symptoms. There are several tests for the <span class="qt0">flu</span>. For the tests, your provider will swipe the inside of your nose or the back of your throat with a swab. Then the swab will be tested for the <span class="qt0">flu</span> virus.</p><p>Some tests are quick and give results in 15-20 minutes. But these tests are not as accurate as other <span class="qt0">flu</span> tests. These other tests can give you the results in one hour or several hours.</p>What are the treatments for the <span class="qt0">flu</span>?<p>Most people with the <span class="qt0">flu</span> recover on their own without medical care. People with mild cases of the <span class="qt0">flu</span> should stay home and avoid contact with others, except to get medical care.</p><p>But if you have symptoms of <span class="qt0">flu</span> and are in a high risk group or are very sick or worried about your illness, contact your health care provider. You might need antiviral medicines to treat your <span class="qt0">flu</span>. Antiviral medicines can make the illness milder and shorten the time you are sick. They also can prevent serious <span class="qt0">flu</span> complications. They usually work best when you start taking them within 2 days of getting sick.</p>Can the <span class="qt0">flu</span> be prevented?<p>The best way to prevent the <span class="qt0">flu is to get a flu</span> vaccine every year. But it's also important to have good health habits like covering your cough and washing your hands often. This can help stop the spread of germs and prevent the <span class="qt0">flu</span>.</p><p>Centers for Disease Control and Prevention</p>

[Agent]: The flu, also called influenza, is a respiratory infection caused by viruses. Each year, millions of Americans get sick with the flu. Sometimes it causes mild illness, but it can also be serious or even deadly, especially for people over 65, newborn babies, and people with certain chronic illnesses.

The flu is caused by flu viruses that spread from person to person. When someone with the flu coughs, sneezes, or talks, they spray tiny droplets. These droplets can land in the mouths or noses of people who are nearby. Less often, a person may get flu by touching a surface or object that has flu virus on it and then touching their own mouth, nose, or possibly their eyes.

Symptoms of the flu come on suddenly and may include:
* Fever or feeling feverish/chills
* Cough
* Sore throat
* Runny or stuffy nose
* Muscle or body aches
* Headaches
* Fatigue (tiredness)

Some people may also have vomiting and diarrhea, which is more common in children.

The best way to prevent the flu is to get a flu vaccine every year. It’s also important to have good health habits like covering your cough and washing your hands often to help stop the spread of germs and prevent the flu.


#### Example 3: NIH MCP Server testing

In [ ]:
await run_lc_react_agent(server_configs, "can you tell me icd-10 code for influenza A?")

[User]: can you tell me icd-10 code for influenza A?

[Agent]: 

-Tool-: Code: J09.X1, Name: Influenza due to identified novel influenza A virus with pneumonia
Code: J09.X3, Name: Influenza due to identified novel influenza A virus with gastrointestinal manifestations
Code: J09.X9, Name: Influenza due to identified novel influenza A virus with other manifestations
Code: J09.X2, Name: Influenza due to identified novel influenza A virus with other respiratory manifestations
Code: A41.3, Name: Sepsis due to Hemophilus influenzae

[Agent]: Here are some possible ICD-10 codes for Influenza A:
J09.X1: Influenza due to identified novel influenza A virus with pneumonia
J09.X3: Influenza due to identified novel influenza A virus with gastrointestinal manifestations
J09.X9: Influenza due to identified novel influenza A virus with other manifestations
J09.X2: Influenza due to identified novel influenza A virus with other respiratory manifestations


#### Agent streaming

In [ ]:
async def run_streaming_agent():
    async with MultiServerMCPClient(
        {
            "nih": {
                "command": "python",
                "args": ["./server/nih.py"],
                "transport": "stdio",
            },
        }
    ) as client:
        agent = create_react_agent(llm, client.get_tools())
        # Initialize conversation history using simple tuples
        inputs = {"messages": []}

        print("Agent is ready. Type 'exit' to quit.")
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Exiting chat.")
                break

            # Append user message to history
            inputs["messages"].append(("user", user_input))

            # call our graph with streaming to see the steps
            async for state in agent.astream(inputs, stream_mode="values"):
                last_message = state["messages"][-1]
                print(last_message)
                last_message.pretty_print()

            # update the inputs with the agent's response
            inputs["messages"] == state["messages"]

In [ ]:
await run_streaming_agent()

Agent is ready. Type 'exit' to quit.
content='can you tell me icd-10 code for influenza A?' additional_kwargs={} response_metadata={} id='aae6f6ee-0e32-4b39-9213-a9d4247a17e2'
================================ Human Message =================================

can you tell me icd-10 code for influenza A?
content='' additional_kwargs={'function_call': {'name': 'get_icd_10_code', 'arguments': '{"query": "influenza A"}'}} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 98, 'candidates_token_count': 13, 'total_token_count': 111, 'prompt_tokens_details': [{'modality': 1, 'token_count': 98}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 13}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.0007394411099644807, 'model_name': 'gemini-2.0-flash-001'} id='run-df4ed000-b4bb-4018-828b-6273d9690950-0' tool_calls=[{'name': 'get_icd_10_code', 'args': {'query': 'influenza A'}, 

### Option 2: Build your own agent to test MCP servers


##### Gemini Agent

Within an MCP client session, this agent loop runs a multi-turn conversation loop with a Gemini model, handling tool calls via MCP server.

This function orchestrates the interaction between a user prompt, a Gemini model capable of function calling, and a session object that provides and executes tools. It handles the cycle of:
-  Gemini gets tool information from MCP client session
-  Sending the user prompt (and conversation history) to the model.
-  If the model requests tool calls, Gemini makes initial function calls to get structured data as per schema, and 
-  Sending the tool execution results back to the model.
-  Repeating until the model provides a text response or the maximum number of tool execution turns is reached.
-  Gemini generates final response based on tool responses and original query.
  
MCP integration with Gemini

<img src="asset/mcp_tool_call.png" alt="MCP with Gemini" height="700">

In [30]:
# --- Configuration ---
# Consider using a more recent/recommended model if available and suitable

DEFAULT_MAX_TOOL_TURNS = 5  # Maximum consecutive turns for tool execution
DEFAULT_INITIAL_TEMPERATURE = (
    0.0  # Temperature for the first LLM call (more deterministic)
)
DEFAULT_TOOL_CALL_TEMPERATURE = (
    1.0  # Temperature for LLM calls after tool use (potentially more creative)
)

# Make tool calls via MCP Server


async def _execute_tool_calls(
    function_calls: List[types.FunctionCall], session: ClientSession
) -> List[types.Part]:
    """
    Executes a list of function calls requested by the Gemini model via the session.

    Args:
        function_calls: A list of FunctionCall objects from the model's response.
        session: The session object capable of executing tools via `call_tool`.

    Returns:
        A list of Part objects, each containing a FunctionResponse corresponding
        to the execution result of a requested tool call.
    """
    tool_response_parts: List[types.Part] = []
    print(f"--- Executing {len(function_calls)} tool call(s) ---")

    for func_call in function_calls:
        tool_name = func_call.name
        # Ensure args is a dictionary, even if missing or not a dict type
        args = func_call.args if isinstance(func_call.args, dict) else {}
        print(f"  Attempting to call session tool: '{tool_name}' with args: {args}")

        tool_result_payload: Dict[str, Any]
        try:
            # Execute the tool using the provided session object
            # Assumes session.call_tool returns an object with attributes
            # like `isError` (bool) and `content` (list of Part-like objects).
            tool_result = await session.call_tool(tool_name, args)
            print(f"  Session tool '{tool_name}' execution finished.")

            # Extract result or error message from the tool result object
            result_text = ""
            # Check structure carefully based on actual `session.call_tool` return type
            if (
                hasattr(tool_result, "content")
                and tool_result.content
                and hasattr(tool_result.content[0], "text")
            ):
                result_text = tool_result.content[0].text or ""

            if hasattr(tool_result, "isError") and tool_result.isError:
                error_message = (
                    result_text
                    or f"Tool '{tool_name}' failed without specific error message."
                )
                print(f"  Tool '{tool_name}' reported an error: {error_message}")
                tool_result_payload = {"error": error_message}
            else:
                print(
                    f"  Tool '{tool_name}' succeeded. Result snippet: {result_text[:150]}..."
                )  # Log snippet
                tool_result_payload = {"result": result_text}

        except Exception as e:
            # Catch exceptions during the tool call itself
            error_message = f"Tool execution framework failed: {type(e).__name__}: {e}"
            print(f"  Error executing tool '{tool_name}': {error_message}")
            tool_result_payload = {"error": error_message}

        # Create a FunctionResponse Part to send back to the model
        tool_response_parts.append(
            types.Part.from_function_response(
                name=tool_name, response=tool_result_payload
            )
        )
    print(f"--- Finished executing tool call(s) ---")
    return tool_response_parts


async def run_agent_loop(
    prompt: str,
    client: genai.Client,
    session: ClientSession,
    model_id: str = MODEL_ID,
    max_tool_turns: int = DEFAULT_MAX_TOOL_TURNS,
    initial_temperature: float = DEFAULT_INITIAL_TEMPERATURE,
    tool_call_temperature: float = DEFAULT_TOOL_CALL_TEMPERATURE,
) -> types.GenerateContentResponse:
    """
    Runs a multi-turn conversation loop with a Gemini model, handling tool calls.

    This function orchestrates the interaction between a user prompt, a Gemini
    model capable of function calling, and a session object that provides
    and executes tools. It handles the cycle of:
    1. Sending the user prompt (and conversation history) to the model.
    2. If the model requests tool calls, executing them via the `session`.
    3. Sending the tool execution results back to the model.
    4. Repeating until the model provides a text response or the maximum
       number of tool execution turns is reached.

    Args:
        prompt: The initial user prompt to start the conversation.
        client: An initialized Gemini GenerativeModel client object

        session: An active session object responsible for listing available tools
                 via `list_tools()` and executing them via `call_tool(tool_name, args)`.
                 It's also expected to have an `initialize()` method.
        model_id: The identifier of the Gemini model to use (e.g., "gemini-1.5-pro-latest").
        max_tool_turns: The maximum number of consecutive turns dedicated to tool calls
                        before forcing a final response or exiting.
        initial_temperature: The temperature setting for the first model call.
        tool_call_temperature: The temperature setting for subsequent model calls
                               that occur after tool execution.

    Returns:
        The final Response from the Gemini model after the
        conversation loop concludes (either with a text response or after
        reaching the max tool turns).

    Raises:
        ValueError: If the session object does not provide any tools.
        Exception: Can potentially raise exceptions from the underlying API calls
                   or session tool execution if not caught internally by `_execute_tool_calls`.
    """
    print(
        f"Starting agent loop with model '{model_id}' and prompt: '{prompt[:100]}...'"
    )

    # Initialize conversation history with the user's prompt
    contents: List[types.Content] = [
        types.Content(role="user", parts=[types.Part(text=prompt)])
    ]

    # Ensure the session is ready (if needed)
    if hasattr(session, "initialize") and callable(session.initialize):
        print("Initializing session...")
        await session.initialize()
    else:
        print("Session object does not have an initialize() method, proceeding anyway.")

    # --- 1. Discover Tools from Session ---
    print("Listing tools from session...")
    # Assumes session.list_tools() returns an object with a 'tools' attribute (list)
    # Each item in the list should have 'name', 'description', and 'inputSchema' attributes.
    session_tool_list = await session.list_tools()

    if not session_tool_list or not session_tool_list.tools:
        raise ValueError("No tools provided by the session. Agent loop cannot proceed.")

    # Convert session tools to the format required by the Gemini API
    gemini_tool_config = types.Tool(
        function_declarations=[
            types.FunctionDeclaration(
                name=tool.name,
                description=tool.description,
                parameters=tool.inputSchema,  # Assumes inputSchema is compatible
            )
            for tool in session_tool_list.tools
        ]
    )
    print(
        f"Configured Gemini with {len(gemini_tool_config.function_declarations)} tool(s)."
    )

    # --- 2. Initial Model Call ---
    print("Making initial call to Gemini model...")
    current_temperature = initial_temperature
    response = await client.aio.models.generate_content(
        model=MODEL_ID,
        contents=contents,  # Send updated history
        config=types.GenerateContentConfig(
            temperature=1.0,
            tools=[gemini_tool_config],
        ),  # Keep sending same config
    )
    print("Initial response received.")

    # Append the model's first response (potentially including function calls) to history
    # Need to handle potential lack of candidates or content
    if not response.candidates:
        print("Warning: Initial model response has no candidates.")
        # Decide how to handle this - raise error or return the empty response?
        return response
    contents.append(response.candidates[0].content)

    # --- 3. Tool Calling Loop ---
    turn_count = 0
    # Check specifically for FunctionCall objects in the latest response part
    latest_content = response.candidates[0].content
    has_function_calls = any(part.function_call for part in latest_content.parts)

    while has_function_calls and turn_count < max_tool_turns:
        turn_count += 1
        print(f"\n--- Tool Turn {turn_count}/{max_tool_turns} ---")

        # --- 3.1 Execute Pending Function Calls ---
        function_calls_to_execute = [
            part.function_call for part in latest_content.parts if part.function_call
        ]
        tool_response_parts = await _execute_tool_calls(
            function_calls_to_execute, session
        )

        # --- 3.2 Add Tool Responses to History ---
        # Send back the results for *all* function calls from the previous turn
        contents.append(
            types.Content(role="function", parts=tool_response_parts)
        )  # Use "function" role
        print(f"Added {len(tool_response_parts)} tool response part(s) to history.")

        # --- 3.3 Make Subsequent Model Call with Tool Responses ---
        print("Making subsequent API call to Gemini with tool responses...")
        current_temperature = tool_call_temperature  # Use different temp for follow-up
        response = await client.aio.models.generate_content(
            model=MODEL_ID,
            contents=contents,  # Send updated history
            config=types.GenerateContentConfig(
                temperature=1.0,
                tools=[gemini_tool_config],
            ),
        )
        print("Subsequent response received.")

        # --- 3.4 Append latest model response and check for more calls ---
        if not response.candidates:
            print("Warning: Subsequent model response has no candidates.")
            break  # Exit loop if no candidates are returned
        latest_content = response.candidates[0].content
        contents.append(latest_content)
        has_function_calls = any(part.function_call for part in latest_content.parts)
        if not has_function_calls:
            print(
                "Model response contains text, no further tool calls requested this turn."
            )

    # --- 4. Loop Termination Check ---
    if turn_count >= max_tool_turns and has_function_calls:
        print(
            f"Maximum tool turns ({max_tool_turns}) reached. Exiting loop even though function calls might be pending."
        )
    elif not has_function_calls:
        print("Tool calling loop finished naturally (model provided text response).")

    # --- 5. Return Final Response ---
    print("Agent loop finished. Returning final response.")
    return response

#### Set up MCP client

In [ ]:
async def run_simple_agent(server_params, query):
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(
            read,
            write,
        ) as session:
            # Test prompt
            prompt = query
            print(f"Running agent loop with prompt: {prompt}")
            # Run agent loop
            res = await run_agent_loop(prompt, client, session)
            return res

In [26]:
bq_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/bq.py"],
)

In [27]:
med_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/med.py"],
)

In [28]:
nih_server_params = StdioServerParameters(
    command="python",
    # Make sure to update to the full absolute path to your server file
    args=["./server/nih.py"],
)

In [ ]:
bq_query = (
    "Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'"
)
bq_res = await run_simple_agent(bq_server_params, bq_query)
print(bq_res.text)

Running agent loop with prompt: Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please list my BigQuery tables, project id is 'dw-genai-dev', location is 'us'...'
Initializing session...
Listing tools from session...
Configured Gemini with 2 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'list_tables' with args: {'project_id': 'dw-genai-dev'}
  Session tool 'list_tables' execution finished.
  Tool 'list_tables' succeeded. Result snippet: demo_dataset1.item_table
demo_dataset1.user_table
demo_dataset2.item_table
demo_dataset2.user_table...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested 

In [ ]:
med_query = "Please explain flu in detail."
med_res = await run_simple_agent(med_server_params, med_query)
print(med_res.text)

Running agent loop with prompt: Please explain flu in detail.
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please explain flu in detail....'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_medical_term' with args: {'term': 'flu'}
  Session tool 'get_medical_term' execution finished.
  Tool 'get_medical_term' succeeded. Result snippet: What is the <span class="qt0">flu</span>?<p>The <span class="qt0">flu</span>, also called <span class="qt0">influenza</span>, is a respiratory infecti...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calling loop finished naturally

In [ ]:
nih_query = "Please tell me icd-10 code for pneumonia"
nih_res = await run_simple_agent(nih_server_params, nih_query)
print(nih_res.text)

Running agent loop with prompt: Please tell me icd-10 code for pneumonia
Starting agent loop with model 'gemini-2.5-pro-exp-03-25' and prompt: 'Please tell me icd-10 code for pneumonia...'
Initializing session...
Listing tools from session...
Configured Gemini with 1 tool(s).
Making initial call to Gemini model...
Initial response received.

--- Tool Turn 1/5 ---
--- Executing 1 tool call(s) ---
  Attempting to call session tool: 'get_icd_10_code' with args: {'query': 'pneumonia'}
  Session tool 'get_icd_10_code' execution finished.
  Tool 'get_icd_10_code' succeeded. Result snippet: Code: A01.03, Name: Typhoid pneumonia
Code: A02.22, Name: Salmonella pneumonia
Code: A54.84, Name: Gonococcal pneumonia
Code: B01.2, Name: Varicella p...
--- Finished executing tool call(s) ---
Added 1 tool response part(s) to history.
Making subsequent API call to Gemini with tool responses...
Subsequent response received.
Model response contains text, no further tool calls requested this turn.
Tool calli